POLARITY - ROBERTA BASE - DATASET ORIGINAL - TITTLE + OPINION

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
# tf.config.set_logical_device_configuration(
# gpus[0],
# [tf.config.LogicalDeviceConfiguration(memory_limit=9216  )])

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [3]:
from datasets import load_dataset, DatasetDict

raw_dataset = load_dataset("javilonso/mex_data_title_with_opinion", use_auth_token=True)

Using custom data configuration javilonso--mex_data_title_with_opinion-b95d308600ebe512
Reusing dataset parquet (/home/javilonso/.cache/huggingface/datasets/parquet/javilonso--mex_data_title_with_opinion-b95d308600ebe512/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
raw_dataset

DatasetDict({
    test: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', 'Title_Opinion', '__index_level_0__'],
        num_rows: 5987
    })
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', 'Title_Opinion', '__index_level_0__'],
        num_rows: 23944
    })
})

In [5]:
def tokenize(example):
    tokenized_example = tokenizer(example["Title_Opinion"], truncation=True)
    tokenized_example["label"] = example["Polarity"]
    return tokenized_example


tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)

Loading cached processed dataset at /home/javilonso/.cache/huggingface/datasets/parquet/javilonso--mex_data_title_with_opinion-b95d308600ebe512/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-f7972f5f1dd67afa.arrow
Loading cached processed dataset at /home/javilonso/.cache/huggingface/datasets/parquet/javilonso--mex_data_title_with_opinion-b95d308600ebe512/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-b71e56d363b28b9c.arrow


In [6]:
# tokenized_dataset["train"][0]

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [8]:
batch_size = 8

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=batch_size,
)

tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=batch_size,
)

In [9]:
model.config.num_labels

5

In [10]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3060, compute capability 8.6


In [11]:
from transformers import create_optimizer

num_epochs = 2
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [12]:
from transformers.keras_callbacks import PushToHubCallback
callback = PushToHubCallback(output_dir="Mex_Rbta_TitleWithOpinion_Polarity", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs
)

/home/javilonso/NLP_Sentiment_UC3M/RoBERTaESP/DATA_TITLE_WITH_OPINION/Mex_Rbta_TitleWithOpinion_Polarity is already a clone of https://huggingface.co/javilonso/Mex_Rbta_TitleWithOpinion_Polarity. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/2
2993/2993 [==============================] - 1112s 371ms/step - loss: 0.3691 - val_loss: 0.5035


Upload file tf_model.h5:   0%|          | 32.0k/476M [00:00<?, ?B/s]

To https://huggingface.co/javilonso/Mex_Rbta_TitleWithOpinion_Polarity
   36d975d..7f871e9  main -> main



In [13]:
# import numpy as np
# from datasets import load_metric
 
# load_accuracy = load_metric("accuracy")
# load_f1 = load_metric("f1")
# for batch in tf_eval_dataset:
#     logits = model.predict(batch)["logits"]
#     labels = batch["labels"]
#     predictions = np.argmax(logits, axis=-1)
#     load_accuracy.add_batch(predictions=predictions, references=labels)
#     load_f1.add_batch(predictions=predictions, references=labels)


# accuracy = load_accuracy.compute()["accuracy"]
# f1 = load_f1.compute(average=None)["f1"]

In [14]:
# print(f'Accuracy: {accuracy:.2f}')
# print(f'F1: {f1}')


------------------- SKLEARN METRICS -------------------

In [15]:
import numpy as np
from datasets import load_metric
y_test_aux = []
y_pred_aux = []
for batch in tf_eval_dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    y_test_aux.append(labels)
    y_pred_aux.append(predictions)

# Flatten arrays
y_pred = []
for arr in y_pred_aux:
    for elem in arr:
        y_pred.append(elem)
y_pred = np.array(y_pred)
        
y_test = []
for arr in y_test_aux:
    for elem in arr:
        y_test.append(elem)
y_test = np.array(y_test)

# TEST Predictions
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.50      0.57       110
           1       0.35      0.33      0.34       136
           2       0.54      0.55      0.55       400
           3       0.55      0.52      0.53      1115
           4       0.89      0.91      0.90      4226

    accuracy                           0.79      5987
   macro avg       0.60      0.56      0.58      5987
weighted avg       0.79      0.79      0.79      5987



In [ ]:
# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# model_name = "classificationEsp1_TitleWithOpinion_Polarity"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [ ]:
# model.push_to_hub("javilonso/classificationEsp1_TitleWithOpinion_Polarity")
# tokenizer.push_to_hub("javilonso/classificationEsp1_TitleWithOpinion_Polarity")

In [ ]:
# # Tests ------

# check_point = "javilonso/classificationEsp1_Augmented_Polarity"

# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
# model = TFAutoModelForSequenceClassification.from_pretrained(check_point)
# tokenizer = AutoTokenizer.from_pretrained(check_point)


# from datasets import load_dataset, DatasetDict

# raw_dataset = DatasetDict.load_from_disk(".././data")

# def tokenize(example):
#     tokenized_example = tokenizer(example["Opinion"], truncation=True)
#     tokenized_example["label"] = example["Polarity"]
#     return tokenized_example


# tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)


# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")


# tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
#     columns=["attenti on_mask", "input_ids", "label"],
#     collate_fn=data_collator,
#     shuffle=True,
#     batch_size=16,
# )

# tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "label"],
#     collate_fn=data_collator,
#     shuffle=False,
#     batch_size=16,
# )

# import numpy as np
# from datasets import load_metric
 
# load_accuracy = load_metric("accuracy")
# load_f1 = load_metric("f1")
# for batch in tf_eval_dataset:
#     logits = model.predict(batch)["logits"]
#     labels = batch["labels"]
#     predictions = np.argmax(logits, axis=-1)
#     load_accuracy.add_batch(predictions=predictions, references=labels)
#     load_f1.add_batch(predictions=predictions, references=labels)


# accuracy = load_accuracy.compute()["accuracy"]
# f1 = load_f1.compute(average=None)["f1"]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at javilonso/classificationEsp1_Attraction.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Loading cached processed dataset at ../data/train/cache-1f28644c3516951f.arrow
Loading cached processed dataset at ../data/test/cache-82babf341eaded68.arrow


In [7]:
# print(f'Accuracy: {accuracy:.4f}')
# print(f'F1: {f1}')


Accuracy: 0.9883
F1: [0.99079755 0.97993411 0.99385343]
